## Create Azure Resources¶

This notebook creates relevant Azure resources. It creates a recource group where an IoT hub with an IoT edge device identity is created. It also creates an Azure container registry (ACR).

In [1]:
from dotenv import set_key, get_key, find_dotenv
from pathlib import Path
import json
import time

To create or access an Azure ML Workspace, you will need the following information:

* An Azure subscription id
* A resource group name
* A region for your resources

We also require you to provide variable names that will be used to create these resources in later notebooks.

In [1]:
# Azure resources
subscription_id = "<subscription_id>"
resource_group = "<resource_group>"  
resource_region = "resource_region" # e.g. resource_region = "eastus"

# IoT hub name - a globally UNIQUE name is required, e.g. iot_hub_name = "myiothubplusrandomnumber".
iot_hub_name = "<iot_hub_name>" 

device_id = "<device_id>" # the name you give to the edge device. e.g. device_id = "mydevice"

# azure container registry  name - a globally UNIQUE name is required, e.g. arc_name = "myacrplusrandomnumber"
acr_name = '<acr_name>' 


Create and initialize a dotenv file for storing parameters used in multiple notebooks.

In [7]:
env_path = find_dotenv()
if env_path == "":
    Path(".env").touch()
    env_path = find_dotenv()

In [ ]:
set_key(env_path, "subscription_id", subscription_id)
set_key(env_path, "resource_group", resource_group)
set_key(env_path, "resource_region", resource_region)

set_key(env_path, "iot_hub_name", iot_hub_name)
set_key(env_path, "device_id", device_id)

set_key(env_path,"acr_name", acr_name)


In [ ]:
acr_login_server = '{}.azurecr.io'.format(acr_name)
set_key(env_path,"acr_login_server", acr_login_server)

## Create Azure Resources


In [ ]:
# login in your account
accounts = !az account list --all -o tsv
if "Please run \"az login\" to access your accounts." in accounts[0]:
    !az login -o table
else:
    print("Already logged in")

Below we will reload it just to make sure that everything is working.

In [ ]:
!az account set --subscription $subscription_id 

In [ ]:
# create a new resource group
!az group create -l $resource_region -n $resource_group

### Create IoT Hub

In [ ]:
# install az-cli iot extension - I had to use "sudo -i" to make it work
!sudo -i az extension add --name azure-cli-iot-ext

In [ ]:
!az iot hub list --resource-group $resource_group -o table

In [ ]:
# Command to create a Standard tier S1 hub with name `iot_hub_name` in the resource group `resource_group`.
!az iot hub create --resource-group $resource_group --name $iot_hub_name --sku S1

In [ ]:
# Command to create a free tier F1 hub. You may encounter error "Max number of Iot Hubs exceeded for sku = Free" if quota is reached.
# !az iot hub create --resource-group $resource_group --name $iot_hub_name --sku F1

### Register an IoT Edge device
In the Azure cloud shell, enter the following command to create a device with name device_id in your iot hub.

In [ ]:
time.sleep(20) # Wait 20 seconds before creating a device
print("az iot hub device-identity create --hub-name {} --device-id {} --edge-enabled -g {}".format(iot_hub_name,device_id,resource_group))

In [ ]:
!az iot hub device-identity create --hub-name $iot_hub_name --device-id $device_id --edge-enabled -g $resource_group

Obtain device_connection_string. It will be used in the next step.

In [ ]:
print("az iot hub device-identity show-connection-string --device-id {} --hub-name {} -g {}".format(device_id, iot_hub_name,resource_group))

In [ ]:
json_data = !az iot hub device-identity show-connection-string --device-id $device_id --hub-name $iot_hub_name -g $resource_group
print(json_data)

In [ ]:
device_connection_string = json.loads(''.join([i for i in json_data if 'WARNING' not in i]))['connectionString']
print(device_connection_string)

In [ ]:
set_key(env_path, "device_connection_string", device_connection_string)

### Create Azure Container Registry

In [ ]:
!az acr create -n $acr_name -g $resource_group --sku Standard --admin-enabled

In [3]:
!az acr login --name $acr_name


Login Succeeded
WARNING! Your password will be stored unencrypted in /home/mylogin/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [ ]:
acr_password = !az acr credential show -n $acr_name --query passwords[0].value
acr_password =  "".join(acr_password)
acr_password = acr_password.strip('\"')

In [ ]:
set_key(env_path,"acr_password", acr_password)


In this notebook, we created relevant Azure resources. We also created a ".env" file to save and reuse the variables needed cross all the notebooks. We can now move on to the next notebook [02_IoTEdgeConfig.ipynb](02_IoTEdgeConfig.ipynb).